In [ ]:
!pip install pymorphy3

In [ ]:
!pip install pyvis

In [ ]:
import csv
import nltk
import pymorphy3
import pandas as pd
from collections import Counter
from pyvis.network import Network
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

# Подготовка данных

In [ ]:
# перед этим загрузить файл all_data.csv
df = pd.read_csv('all_data.csv', encoding='utf-8')
target_columns = [
    'Запишите, пожалуйста, вступление, которое вы слышали/использовали чаще всего',
    'Запишите, пожалуйста, другой вариант вступления, который вы слышали',
    'Запишите, пожалуйста, ещё один вариант, если вы слышали какую-то третью вариацию вступления'
]
df_filtered = df[target_columns].copy()
df_filtered.columns = ['intro1', 'intro2', 'intro3']
all_entries = pd.concat([
    df_filtered['intro1'],
    df_filtered['intro2'],
    df_filtered['intro3']
], ignore_index=True)
all_entries_clean = all_entries.dropna()
all_entries_clean = all_entries_clean.astype(str).str.strip()

In [ ]:
# файл с непочищенными вступлениями
all_entries_clean.to_csv('intro_uncleaned.csv', index=False, encoding='utf-8')

In [ ]:
all_entries_clean = all_entries_clean.str.lower().str.replace(r'[^\w\s]', '', regex=True)
df_result = pd.DataFrame({'intro': all_entries_clean})
df_result = df_result[df_result['intro'] != '']
df_sorted_asc = df_result.sort_values('intro', ascending=True)

In [ ]:
# файл без знаков препинания, отсортированный
df_sorted_asc.to_csv('intro.csv', index=False, encoding='utf-8')

In [ ]:
df_result.head(10)

# Лемматизация и нахождение пар

In [ ]:
def lemmatize_and_count_pairs(input_file, output_file):
    morph = pymorphy3.MorphAnalyzer()
    pairs_counter = Counter()

    with open(input_file, 'r', encoding='utf-8') as file:
        for line in file:
            words = [word.strip() for word in line.strip().split() if word.strip()]
            lemmas = []
            for word in words:
                parsed = morph.parse(word.lower())[0]
                lemmas.append(parsed.normal_form)
            for i in range(len(lemmas) - 1):
                pair = (lemmas[i], lemmas[i + 1])
                pairs_counter[pair] += 1

    with open(output_file, 'w', encoding='utf-8', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['word1', 'word2', 'count'])
        sorted_pairs = sorted(pairs_counter.items(), key=lambda x: x[1], reverse=True)
        for (word1, word2), count in sorted_pairs:
            writer.writerow([word1, word2, count])

In [ ]:
# перед этим загрузить просмотренный вручную файл intro_cleaned.csv
if __name__ == '__main__':
    lemmatize_and_count_pairs('intro_cleaned.csv', 'pairs_lemmatized.csv')

# Построение графов

In [ ]:
# перед этим загрузить файл с исправленными леммами pairs_lemmatized_cleaned.csv
# строим граф на всех данных
def network_from_all_pairs(pairs_file, output_html='intro_network_all.html'):
    df = pd.read_csv(pairs_file)
    net = Network(notebook=True, height='750px', width='100%', bgcolor='#222222', font_color='white')
    for _, row in df.iterrows():
        net.add_node(row['word1'], title=row['word1'], size=15)
        net.add_node(row['word2'], title=row['word2'], size=15)
        net.add_edge(row['word1'], row['word2'], value=row['count'], title=f'Частота: {row['count']}')
    net.force_atlas_2based(gravity=-100, central_gravity=0.01, spring_length=100)
    net.show(output_html)

In [ ]:
network_from_all_pairs('pairs_lemmatized_cleaned.csv')

In [ ]:
# строим граф с ограничением частотности >= 2
def network_from_pairs(pairs_file, output_html='intro_network_2andmore.html'):
    df = pd.read_csv(pairs_file)
    df = df[df['count'] >= 2]
    net = Network(notebook=True, height='750px', width='100%', bgcolor='#222222', font_color='white')
    for _, row in df.iterrows():
        net.add_node(row['word1'], title=row['word1'], size=15)
        net.add_node(row['word2'], title=row['word2'], size=15)
        net.add_edge(row['word1'], row['word2'], value=row['count'], title=f"Частота: {row['count']}")
    net.force_atlas_2based(gravity=-100, central_gravity=0.01, spring_length=100)
    net.show(output_html)

In [ ]:
network_from_pairs('pairs_lemmatized_cleaned.csv')

In [ ]:
# строим граф без стоп-слов с ограничением частотности
def clean_stopwords(input_file, output_file):
    stop = set(stopwords.words('russian'))
    clean_pairs = []
    with open(input_file, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            word1 = row['word1']
            word2 = row['word2']
            count = int(row['count'])
            # Пропускаем если оба слова не стоп-слова
            if word1 not in stop and word2 not in stop:
                clean_pairs.append((word1, word2, count))
    clean_pairs.sort(key=lambda x: x[2], reverse=True)
    with open(output_file, 'w', encoding='utf-8', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['word1', 'word2', 'count'])
        writer.writerows(clean_pairs)
    return clean_pairs

In [ ]:
if __name__ == "__main__":
    clean_pairs = clean_stopwords('pairs_lemmatized_cleaned.csv', 'pairs_cleaned.csv')

In [ ]:
def network_from_pairs_stop(pairs_file, output_html='intro_network_stop.html'):
    df = pd.read_csv(pairs_file)
    df = df[df['count'] >= 2]
    net = Network(notebook=True, height='750px', width='100%', bgcolor='#222222', font_color='white')
    for _, row in df.iterrows():
        net.add_node(row['word1'], title=row['word1'], size=15)
        net.add_node(row['word2'], title=row['word2'], size=15)
        net.add_edge(row['word1'], row['word2'], value=row['count'], title=f'Частота: {row['count']}')
    net.force_atlas_2based(gravity=-100, central_gravity=0.01, spring_length=100)
    net.show(output_html)

In [ ]:
network_from_pairs_stop('pairs_cleaned.csv')